In [ ]:
!pip install openai

In [ ]:
!pip install langchain

In [ ]:
!pip install unstructured

In [ ]:
!pip install tiktoken

In [ ]:
pip install faiss-cpu

In [ ]:
from langchain.document_loaders import UnstructuredPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import OpenAI
from langchain.callbacks import get_openai_callback
from langchain.chains.question_answering import load_qa_chain

In [ ]:
loader = UnstructuredPDFLoader('./MDMA_IB.pdf')

In [ ]:
data = loader.load()

In [ ]:
len(data)

In [ ]:
len(data[0].page_content) / 1800

In [ ]:
text_splitter = CharacterTextSplitter(separator="\n", chunk_size=1000, chunk_overlap=20, length_function=len)

In [ ]:
chunks = text_splitter.split_text(data[0].page_content)

In [ ]:
len(chunks)

In [ ]:
with open('secret', 'r') as fr:
    secret = fr.readline().strip()

In [ ]:
embeddings = OpenAIEmbeddings(openai_api_key=secret)

In [ ]:
knowledge_base = FAISS.from_texts(chunks, embeddings)

In [ ]:
knowledge_base

In [ ]:
user_question = "Summarize the results for efficacy for primary endpoints across MAPS studies."
docs = knowledge_base.similarity_search(user_question, k=5)
docs

In [ ]:
llm = OpenAI(openai_api_key=secret)

In [ ]:
chain = load_qa_chain(llm, chain_type="stuff")
with get_openai_callback() as cb:
    response = chain.run(input_documents=docs, question=user_question)
print(cb)

In [ ]:
response

In [ ]:
!mkdir 'maps'

In [ ]:
knowledge_base.save_local(folder_path='./maps/', index_name='maps')